<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/RAG_Application_Using_LlamaIndex_%26_Long_Context_Model_Jamba_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install llama-index
!pip install -U ai21
!pip install llama-index-llms-ai21

  Using cached ai21-3.0.1-py3-none-any.whl.metadata (21 kB)
Using cached ai21-3.0.1-py3-none-any.whl (59 kB)
  Attempting uninstall: ai21
    Found existing installation: ai21 2.15.2
    Uninstalling ai21-2.15.2:
      Successfully uninstalled ai21-2.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-ai21 0.4.0 requires ai21<3.0.0,>=2.13.0, but you have ai21 3.0.1 which is incompatible.
  Using cached ai21-2.15.2-py3-none-any.whl.metadata (22 kB)
Using cached ai21-2.15.2-py3-none-any.whl (88 kB)
  Attempting uninstall: ai21
    Found existing installation: ai21 3.0.1
    Uninstalling ai21-3.0.1:
      Successfully uninstalled ai21-3.0.1


In [8]:
!pip install --upgrade llama-index
!pip install llama-index-embeddings-huggingface

In [17]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ai21 import AI21
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from google.colab import userdata

# Use Hugging Face embeddings
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a directory for data storage
os.makedirs("data", exist_ok=True)

# Download the 10-K filings using wget
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf' -O 'data/amazon_2023.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf' -O 'data/amazon_2022.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/f965e5c3-fded-45d3-bbdb-f750f156dcc9.pdf' -O 'data/amazon_2021.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/336d8745-ea82-40a5-9acc-1a89df23d0f3.pdf' -O 'data/amazon_2020.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/4d39f579-19d8-4119-b087-ee618abf82d6.pdf' -O 'data/amazon_2019.pdf'

# Load the downloaded files
file_list = [os.path.join("data", f) for f in os.listdir("data")]
amzn_10k_docs = SimpleDirectoryReader(input_files=file_list).load_data()

# Create an index with Hugging Face embeddings
index = VectorStoreIndex.from_documents(documents=amzn_10k_docs, embed_model=embed_model)

--2024-12-27 14:45:08--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 3.165.166.61, 3.165.166.194, 3.165.166.67, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|3.165.166.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800598 (782K) [application/pdf]
Saving to: ‘data/amazon_2023.pdf’

data/amazon_2023.pd 100%[===================>] 781.83K  --.-KB/s    in 0.05s   

2024-12-27 14:45:09 (16.7 MB/s) - ‘data/amazon_2023.pdf’ saved [800598/800598]

--2024-12-27 14:45:09--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 3.165.166.61, 3.165.166.194, 3.165.166.67, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|3.165.166.61|:443... connected.
HTTP request sent, 

In [20]:
# Set API keys for the required services
os.environ['AI21_API_KEY'] = userdata.get('AI21_API_KEY')

In [21]:
# Initialize the LLM
llm = AI21(
    model='jamba-instruct',
    temperature=0,
    max_tokens=2000,
)
# Build a query engine
default_query_engine = index.as_query_engine(llm=llm)

In [22]:
# Query the engine
answer = default_query_engine.query("What was the company's revenue in 2021?")
print(answer.response)

The company's revenue in 2021 was not explicitly stated in the provided text.


In [23]:
answer = default_query_engine.query("What was the company's revenue in 2023?")
print(answer.response)

The company's revenue in 2023 was not provided in the given text.


In [24]:
# Large amount of chunks in the retrieval process
extended_query_engine = index.as_query_engine(llm,
                                              similarity_top_k=100)

answer = extended_query_engine.query("What was the company's revenue in 2023?")
print(answer.response)

The company's revenue in 2023 was $574,785 million.


In [25]:
answer = default_query_engine.query("Was there a stock split in the last five years?")
print(answer.response)

No, there has been no stock split in the last five years.


In [26]:
answer = extended_query_engine.query("Was there a stock split in the last five years?")
print(answer.response)

No, there was no stock split in the last five years.
